In [164]:
from loading_data import queryDB
import pandas as pd

In [165]:
import json
import numpy as np
import copy
import os

config_path = 'config.json'

with open(config_path) as config_buffer:
   config = json.loads(config_buffer.read())

model_path = config['model_weights']
num_concepts = len(config['conceptids'])
class_map_file = config['class_map']
concepts = config['conceptids']
classmap = pd.read_csv(class_map_file, header=None).to_dict()[0]

In [166]:
queryDB("select * from videos where filename='DocRicketts-0701_20141216T212020Z_00-48-12-26TC_h264.mp4'")

,id,filename,gpsstart,gpsstop,starttime,endtime,startdepth,enddepth,description
0,8,DocRicketts-0701_20141216T212020Z_00-48-12-26T...,"(36.798367,-122.108671)","(36.798215,-122.108195)",2014-12-16 21:20:20,2014-12-16 21:36:20,217,149,


In [167]:
annotations = queryDB('select * from annotations where videoid=8 and userid!=17')

In [168]:
annotations.head()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,433,8,6,236,178.732221,575.006251,49.006271,698.999957,155.993774,1280.0,720.0,2018-09-05,433.png,433_box.png,None,False,433,None
1,471,8,6,236,192.175752,326.006221,25.006256,443.006213,133.006248,1280.0,720.0,2018-09-05,471.png,471_box.png,None,False,471,None
2,488,8,6,383,207.099977,81.006251,162.006241,417.006221,432.999985,1280.0,720.0,2018-09-05,488.png,488_box.png,None,False,488,None
3,427,8,6,236,176.381214,740.006190,56.006271,842.999934,156.993767,1280.0,720.0,2018-09-05,427.png,427_box.png,None,False,427,None
4,434,8,6,236,178.485498,1116.006251,50.006271,1239.999995,156.993767,1280.0,720.0,2018-09-05,434.png,434_box.png,None,False,434,None


In [169]:
def compute_overlap(annotationA, annotationB):
    # if there is no overlap in x dimension
    if annotationB.x2 - annotationA.x1 < 0 or annotationA.x2 - annotationB.x1 < 0:
        return 0
    # if there is no overlap in y dimension
    if annotationB.y2 - annotationA.y1 < 0 or annotationA.y2 - annotationB.y1 < 0:
        return 0
    
    areaA = (annotationA.x2-annotationA.x1) * (annotationA.y2-annotationA.y1)
    areaB = (annotationB.x2-annotationB.x1) * (annotationB.y2-annotationB.y1)

    width = min(annotationA.x2,annotationB.x2) - min(annotationA.x1,annotationB.x1)
    height = min(annotationA.y2,annotationB.y2) - min(annotationA.y1,annotationB.y1)
    
    area_intersect = height * width
    iou = area_intersect / (areaA + areaB - area_intersect)
    
    return iou

In [170]:
a = annotations.iloc[0]
a.x1 = -1
a.x2 = 0
if compute_overlap(a,annotations.iloc[0]) != 0:
    print("ERROR")
    
a = annotations.iloc[0]
a.x1 = 0
a.x2 = 1000
a.y1 = 0
a.y2 = 1000
b = annotations.iloc[0]
b.x1 = 0
b.x2 = 1000
b.y1 = 0
b.y2 = 500

if compute_overlap(a,b) != .5:
    print("ERROR")

In [203]:
fps = 29.97002997002997

In [171]:
def score_predictions(validation,predictions,iou_thresh, concepts, fps):
    # group predictions by video frames
    predictions = predictions.groupby(['videoid','frame_num'], sort=False)
    predictions = [df for _, df in predictions]
    
    # mapping annotations from prediction list to video/time
    frame_data = {}
    for i, group in enumerate(predictions):
        frame_num = group.iloc[0]['frame_num']
        video = group.iloc[0]['videoid']
        frame_data[(video,frame_num)] = i
    
    # group validation annotations by video frames
    validation = validation.groupby(['videoid','timeinvideo'], sort=False)
    validation = [df for _, df in validation]
    
    # initialize counters for each concept
    true_positives = dict(zip(concepts,[0] * len(concepts)))
    false_positives = dict(zip(concepts,[0] * len(concepts)))
    false_negatives = dict(zip(concepts,[0] * len(concepts)))
    
    # get true and false positives for each frame of validation data
    for group in validation:
        #get time and video for this set
        frame_num = int(round(0,group.iloc[0]['timeinvideo'] * fps))
        video = group.iloc[0]['videoid']
        
        #get corresponding predictions for this set
        try:
            predicted = predictions[frame_data[(video,frame_num)]]
        except:
            print("prediction not found")
            continue
            
        #go through predictions and identify false positives/false negatives/ true_positives
        detected_predictions = []
        detected_truths = dict(zip(concepts,[0] * len(concepts)))
        for index, truth in group.iterrows():
            for index, prediction in predicted.iterrows():
                if prediction.id in detected_predictions or compute_overlap(truth,prediction) < iou_thresh or prediction.conceptid != truth.conceptid:
                    continue
                detected_predictions.append(prediction.id)
                if truth.conceptid == prediction.conceptid:
                    true_positives[prediction.conceptid] += 1
                    detected_truths[prediction.conceptid] += 1
                else:
                    print("HERE")
                    print(truth)
                    print(prediction)
                    false_positives[prediction.conceptid] += 1

        for id in concepts:
            false_negatives[id] += len(group.loc[group['conceptid'] == id]) - detected_truths[id]
        #count number of annotations that were correct, wrong, and missed
        
    f1 = dict(zip(concepts,[0] * len(concepts)))
    for id in concepts:
        try:
            recall = true_positives[id] / (true_positives[id] + false_negatives[id])
            precision = true_positives[id] / (true_positives[id] + false_positives[id])
            f1[id] = 2*recall*precision / (precision+recall)
        except:
            f1[id] = 0
    
    return f1

In [172]:
score_predictions(annotations,annotations,.9,annotations.conceptid.unique())

prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction no

{236: 0, 237: 0, 383: 0, 812: 0, 889: 0, 1009: 0, 2101: 0, 3100: 0}

In [173]:
#DOESNT WORK IF VAL IS MISSING

In [174]:
#is it false positive if something is misidentified, or false positive as well as false negative?
# currently both!

In [175]:
queryDB('select * from annotations limit 1')

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,2178619,19,17,2136,773.44,614.0,474.0,690.0,512.0,1280.0,720.0,2019-01-27,19_773.44_ai.png,6945_773.44_box_ai.png,None,False,6945,23180


In [176]:
queryDB('select * from annotations where userid != 17 limit 1')

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,11107,29,11,1083,96.110808,82.0,285.0,145.984383,321.0,1600.0,900.0,2018-11-13,11107.png,11107_box.png,,False,11107,None


In [177]:
annotations.tail()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
129,552,8,6,1009,382.113777,968.006251,79.006256,1178.006236,355.006256,1280.0,720.0,2018-09-25,1537902851583,1537902851583_box,None,False,552,None
130,553,8,6,383,384.512926,560.006251,0.000000,947.006190,285.993744,1280.0,720.0,2018-09-25,1537902872581,1537902872581_box,None,False,553,None
131,554,8,6,383,395.718291,823.006251,46.006256,1210.006221,332.000000,1280.0,720.0,2018-09-25,1537902894130,1537902894130_box,None,False,554,None
132,556,8,6,237,411.169361,0.000000,0.000000,190.987488,187.987488,1280.0,720.0,2018-09-25,1537902963374,1537902963374_box,None,False,556,None
133,555,8,6,383,403.672527,275.006251,19.006256,497.999965,272.993759,1280.0,720.0,2018-09-25,1537902912565,1537902912565_box,None,False,555,None


In [178]:
results = pd.read_csv('../../test/results.csv')

In [179]:
len(results.objectid.value_counts())

52

In [180]:
len(annotations)

134

In [181]:
results.timeinvideo = results.timeinvideo+160

In [182]:
results

,Unnamed: 0,x1,y1,x2,y2,label,confidence,objectid,timeinvideo
0,0,477.479370,0.000000,640.000000,128.515671,3.0,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.00
1,1,477.000000,0.000000,640.000000,129.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.03
2,2,476.000000,3.000000,639.000000,132.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.07
3,3,475.000000,8.000000,638.000000,137.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.10
4,4,472.000000,11.000000,635.000000,140.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.13
5,5,471.000000,16.000000,634.000000,145.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.17
6,6,468.000000,19.000000,631.000000,148.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.20
7,7,467.000000,24.000000,630.000000,153.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.23
8,8,464.000000,29.000000,627.000000,158.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.27
9,9,463.000000,34.000000,626.000000,163.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.30


In [183]:
label = None
objects = results.groupby(['objectid'])
for oid, result in objects:
    scores = {}
    for k , label in result.groupby(['label']):
        scores[k] = label.confidence.mean()
    idmax = max(scores.keys(), key=(lambda k: scores[k]))
    results.loc[results.objectid == oid,'label'] = idmax

In [184]:
results

,Unnamed: 0,x1,y1,x2,y2,label,confidence,objectid,timeinvideo
0,0,477.479370,0.000000,640.000000,128.515671,3.0,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.00
1,1,477.000000,0.000000,640.000000,129.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.03
2,2,476.000000,3.000000,639.000000,132.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.07
3,3,475.000000,8.000000,638.000000,137.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.10
4,4,472.000000,11.000000,635.000000,140.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.13
5,5,471.000000,16.000000,634.000000,145.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.17
6,6,468.000000,19.000000,631.000000,148.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.20
7,7,467.000000,24.000000,630.000000,153.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.23
8,8,464.000000,29.000000,627.000000,158.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.27
9,9,463.000000,34.000000,626.000000,163.000000,3.0,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.30


In [185]:
class_map_file = config['class_map']
concepts = config['conceptids']

results['label'] = results['label'].apply(lambda x: concepts[int(x)])

In [186]:
temp = list(results.columns)
temp[0] = 'id'
results.columns = temp

In [187]:
results['conceptid'] = results['label']

In [188]:
results

,id,x1,y1,x2,y2,label,confidence,objectid,timeinvideo,conceptid
0,0,477.479370,0.000000,640.000000,128.515671,383,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.00,383
1,1,477.000000,0.000000,640.000000,129.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.03,383
2,2,476.000000,3.000000,639.000000,132.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.07,383
3,3,475.000000,8.000000,638.000000,137.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.10,383
4,4,472.000000,11.000000,635.000000,140.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.13,383
5,5,471.000000,16.000000,634.000000,145.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.17,383
6,6,468.000000,19.000000,631.000000,148.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.20,383
7,7,467.000000,24.000000,630.000000,153.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.23,383
8,8,464.000000,29.000000,627.000000,158.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.27,383
9,9,463.000000,34.000000,626.000000,163.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.30,383


In [189]:
results['videoid'] = 8
score_predictions(annotations,results,.5, concepts)

prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction not found
prediction no

{236: 0.3157894736842105, 383: 0, 1133: 0, 1210: 0, 1629: 0}

In [192]:
results.head(10)

,id,x1,y1,x2,y2,label,confidence,objectid,timeinvideo,conceptid,videoid
0,0,477.47937,0.0,640.0,128.515671,383,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.00,383,8
1,1,477.00000,0.0,640.0,129.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.03,383,8
2,2,476.00000,3.0,639.0,132.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.07,383,8
3,3,475.00000,8.0,638.0,137.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.10,383,8
4,4,472.00000,11.0,635.0,140.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.13,383,8
5,5,471.00000,16.0,634.0,145.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.17,383,8
6,6,468.00000,19.0,631.0,148.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.20,383,8
7,7,467.00000,24.0,630.0,153.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.23,383,8
8,8,464.00000,29.0,627.0,158.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.27,383,8
9,9,463.00000,34.0,626.0,163.000000,383,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,160.30,383,8


In [193]:
annotations.head(10)

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,433,8,6,236,178.732221,575.006251,49.006271,698.999957,155.993774,1280.0,720.0,2018-09-05,433.png,433_box.png,None,False,433,None
1,471,8,6,236,192.175752,326.006221,25.006256,443.006213,133.006248,1280.0,720.0,2018-09-05,471.png,471_box.png,None,False,471,None
2,488,8,6,383,207.099977,81.006251,162.006241,417.006221,432.999985,1280.0,720.0,2018-09-05,488.png,488_box.png,None,False,488,None
3,427,8,6,236,176.381214,740.006190,56.006271,842.999934,156.993767,1280.0,720.0,2018-09-05,427.png,427_box.png,None,False,427,None
4,434,8,6,236,178.485498,1116.006251,50.006271,1239.999995,156.993767,1280.0,720.0,2018-09-05,434.png,434_box.png,None,False,434,None
5,515,8,6,383,221.858010,1012.006251,324.006241,1279.000000,664.993759,1280.0,720.0,2018-09-25,1537835589399,1537835589399_box,None,False,515,None
6,517,8,6,383,234.685132,631.006251,147.006256,808.999995,304.993744,1280.0,720.0,2018-09-25,1537835622221,1537835622221_box,None,False,517,None
7,519,8,6,383,238.310152,619.006251,72.006256,881.993739,277.993744,1280.0,720.0,2018-09-25,1537835661367,1537835661367_box,None,False,519,None
8,420,8,6,236,172.134367,1049.006251,203.006271,1179.993739,317.006248,1280.0,720.0,2018-09-05,420.png,420_box.png,None,False,420,None
9,422,8,6,236,172.134367,833.006251,2.006256,963.993739,116.006248,1280.0,720.0,2018-09-05,422.png,422_box.png,None,False,422,None


In [202]:
a = 32/29.97
a

1.0677344010677345

In [204]:
round(a,0)

1.0